In [1]:
from utils import spark_session, save_table

session = spark_session("purchase-suppression-tmp-ol")

24/11/06 13:08:30 INFO SecurityManager: Changing view acls to: olivyatan,b_perso,*
24/11/06 13:08:30 INFO SecurityManager: Changing modify acls to: olivyatan,b_perso
24/11/06 13:08:30 INFO SecurityManager: Changing view acls groups to: 
24/11/06 13:08:30 INFO SecurityManager: Changing modify acls groups to: 
24/11/06 13:08:30 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(olivyatan, b_perso, *); groups with view permissions: Set(); users  with modify permissions: Set(olivyatan, b_perso); groups with modify permissions: Set()
24/11/06 13:08:30 INFO deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
24/11/06 13:08:30 INFO SparkContext: Running Spark version 3.1.1.0.1.0
24/11/06 13:08:30 INFO ResourceUtils: ==============================================================
24/11/06 13:08:30 INFO ResourceUtils: No custom resources configured for spark.driver.
24/11/06 13:08:30 INFO ResourceUtil

# Configuration

In [2]:
config = dict()
config["filters"] = {
    "start_dt": "2023-07-01",
    "end_dt": "2024-07-01",
    "site_ids": [0],
}

config['event_limits'] = {
    'max_purchases': 1000,
    'max_vi': 1000,
    'max_searches': 1000,
    'max_watches': 1000,
    'max_events': 1000  
}

config['sampling'] = {
    'max_samples': 100000,
    'sample_ratio': 0.1,
    'ref_dt': '2024-05-01'
}

# Sample Datapoints

In [8]:
from user_sample import sample_data

df_sample = sample_data(session, config)
save_table(session, df_sample, "bx_purchase_suppression_samples_1")

# Extract data

In [9]:
from pyspark_query import get_minimal_vi_histories, get_minimal_purchase_histories, get_leaf_cat_data

df = session.table("bx_purchase_suppression_samples_1")
df_users = df.select("BUYER_ID").dropDuplicates().cache()

# #for data point obtain the item's metadata
df_dpoints = get_leaf_cat_data(session, config, df)
save_table(session, df_dpoints, "bx_ps_samples_enriched_1")#df_smp

#for each user obtain purchase history
df_prch_hist = get_minimal_purchase_histories(session, config, df_users)
save_table(session, df_prch_hist, "bx_ps_user_prch_hist_1")#df_prch

#for each user obtain vi history
df_vi_hist = get_minimal_vi_histories(session, config, df_users)
save_table(session, df_vi_hist, "bx_ps_user_vi_hist_1")#df_vi

24/10/01 07:57:31 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

#### Add watches , searches

In [4]:
import features as F

In [3]:
df = session.table("bx_purchase_suppression_samples_1")
df_users = df.select("BUYER_ID").dropDuplicates().cache()

In [5]:
from pyspark_query import get_minimal_watch_histories

In [6]:
df_wtch_hist = get_minimal_watch_histories(session, config, df_users)

In [7]:
df_wtch_hist.count()

13707413

In [9]:
#save_table(session, df_wtch_hist, "bx_ps_user_watch_hist_1")#df_watch

In [18]:
#from pyspark.sql import functions as F
#from pyspark.sql.functions import udf, first, lit, split, explode
#import pyspark_query as pq

def get_minimal_watch_histories(session, config, df_users_to_consider):
    # Get watch histories from bbowac_event_fact with event_type_desc='Watch'
    df_user_histories = session.table("access_views.bbowac_event_fact") \
        .where((F.col("session_start_dt") >= config['filters']['start_dt']) &  
               (F.col("session_start_dt") <= config['filters']['end_dt']) &
               (F.upper(F.col("site_id")).isin(config['filters']['site_ids'])) &  
               (F.col("event_type_desc") == 'Watch') &  # Filter where event_type_desc is 'Watch'
               (F.col("bot_session") == 0))  # Exclude bot sessions

    # Rename columns
    df_user_histories = df_user_histories \
        .withColumnRenamed('buyer_id', 'BUYER_ID') \
        .withColumnRenamed('site_id', 'SITE_ID') \
        .withColumnRenamed('item_id', 'ITEM_ID') \
        .withColumnRenamed('variation_id', 'VARIATION_ID') \
        .withColumnRenamed('leaf_categ_id', 'LEAF_CATEG_ID')  \
        .withColumnRenamed('event_timestamp', 'EVENT_TIMESTAMP')  \
        .select('EVENT_TIMESTAMP', 'BUYER_ID', 'ITEM_ID', 'VARIATION_ID', 'LEAF_CATEG_ID', 'SITE_ID') 
    
    df_user_histories = df_user_histories.join(df_users_to_consider, "BUYER_ID") \
        .select('EVENT_TIMESTAMP', 'BUYER_ID', 'ITEM_ID')

    # Limit history length using max_watches
    df_user_histories = pq.limit_history_len(config['event_limits']['max_watches'],  
                                          config['event_limits']['max_events'],
                                          df_user_histories) 
    
    df_item_ids = df_user_histories.select("ITEM_ID").dropDuplicates() 
    
    df_item_fact = session.table("access_views.DW_LSTG_ITEM") \
        .select("AUCT_TITL", "ITEM_ID", "AUCT_END_DT", "LEAF_CATEG_ID", "ITEM_SITE_ID") \
        .withColumnRenamed("ITEM_SITE_ID", "SITE_ID") \
        .withColumnRenamed("AUCT_TITL", "TITLE") \
        .where(F.col('AUCT_END_DT') >= config['filters']['start_dt']) \
        .join(df_item_ids, "ITEM_ID") \
        .dropDuplicates(["ITEM_ID"])
    
    df_item_fact = pq.lowercase_string_columns(df_item_fact) 
   
    df_final = df_user_histories \
        .join(df_item_fact.select('ITEM_ID', 'TITLE', 'SITE_ID', "LEAF_CATEG_ID"), on='ITEM_ID') \
        .join(pq.get_category_names(session, df_item_fact), on=["SITE_ID", "LEAF_CATEG_ID"]) \
        .withColumn('EVENT_TYPE', lit('WATCH')) 
   
    df_final_watch = df_final.filter(F.col('ITEM_ID').isNotNull() & 
                                  F.col('SITE_ID').isNotNull() &
                                  F.col('LEAF_CATEG_ID').isNotNull() & 
                                  F.col('TITLE').isNotNull() &
                                  F.col('LEAF_CATEG_NAME').isNotNull())

    return pq.lowercase_string_columns(df_final_watch)


In [7]:
#from pyspark_query import get_minimal_watch_histories_ol

In [8]:
#for each user obtain search history
#df_wtch_hist = get_minimal_watch_histories_ol(session, config, df_users)

In [9]:
#for each user obtain search history
df_wtch_hist = get_minimal_watch_histories(session, config, df_users)

In [10]:
#save_table(session, df_wtch_hist, "bx_ps_user_wtch_hist_1")

In [11]:
df_wtch_hist.count()

1868526459

In [12]:
df_wtch_hist.first()

Row(EVENT_TIMESTAMP=datetime.datetime(2023, 12, 3, 3, 54, 9, 201000), BUYER_ID=Decimal('112441611'), LEAF_CATEG_ID=Decimal('25815'), ITEM_ID=Decimal('115963551824'), VARIATION_ID=None, SITE_ID=Decimal('0'), EVENT_TYPE='watch', LEAF_CATEG_NAME='home & garden:bedding:bedding accessories', CATEG_LVL2_ID=Decimal('20444'), META_CATEG_ID=Decimal('11700'), CATEG_LVL2_NAME='bedding', META_CATEG_NAME='home & garden')

In [3]:
df_srch_hist = session.table("bx_ps_user_srch_hist_1")

In [4]:
df_vi_hist = session.table("bx_ps_user_vi_hist_1")

In [ ]:
df_wtch_hist.count()

In [6]:
df_vi_hist.count()

53493087

In [5]:
df_srch_hist.count()

33368617

In [ ]:
if session:
    session.stop()

In [7]:
df_srch_hist.first()

Row(EVENT_TIMESTAMP=datetime.datetime(2023, 10, 18, 19, 3, 49, 585000), BUYER_ID=Decimal('183988313'), LEAF_CATEG_ID=25815, TITLE='sheet straps', SITE_ID=Decimal('0'), EVENT_TYPE='search', LEAF_CATEG_NAME='home & garden:bedding:bedding accessories', CATEG_LVL2_ID=Decimal('20444'), META_CATEG_ID=Decimal('11700'), CATEG_LVL2_NAME='bedding', META_CATEG_NAME='home & garden')